# 02807 Computational Tools for Data Science
##### November 28, 2023

In [1]:
import os
import math
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap
from IPython.display import clear_output
from collections import Counter
import nltk
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
import warnings
from wordcloud import WordCloud
import datetime
warnings.filterwarnings("ignore")

## Web scraping

In [2]:
def scrape_guardian_article(df_row):
    # Get soup
    soup = df_row['soup']

    # Attempt to get summary
    summary_tag = soup.find('div', {'data-gu-name': 'standfirst'})
    summary = summary_tag.get_text(strip=True) if summary_tag else None

    # Attempt to get author
    author_tag = soup.find('a', rel='author')
    author = author_tag.get_text(strip=True) if author_tag else 'Unknown'

    # Attempt to get content
    content_tags = soup.find_all('p')
    content = ' '.join(tag.get_text(strip=True) for tag in content_tags[1:])

    # Attempt to get date
    date_tag = soup.find('span', class_='dcr-u0h1qy')
    date_string = date_tag.get_text(strip=True) if date_tag else 'Unknown'

    """
    # Parse the date if it's not 'Unknown'
    if date_string != 'Unknown':
        try:
            date_format = "%B %d, %Y %I:%M%p"
            date = datetime.strptime(date_string, date_format)
        except ValueError:
            date = 'Unknown'
    else:
        date = 'Unknown'

    """

    # Update the df_row with the scraped data
    df_row['summary'] = summary
    df_row['author'] = author
    df_row['content'] = content
    #df_row['date'] = date
    df_row['date'] = date_string

    return df_row

## Scraper function to scrape articles from The Washington Post
def scrape_washington_post_article(df_row):
    # Get soup
    soup = df_row['soup']

    # Attempt to get summary
    summary = None
    summary_candidates = ['PJLV PJLV-iPJLV-css grid-center w-100']
    for candidate in summary_candidates:
        summary_tag = soup.find(class_=candidate)
        if summary_tag:
            summary = summary_tag.get_text(strip=True)
            break

    # Attempt to get author
    author = None
    author_candidates = ['wpds-c-cNdzuP wpds-c-cNdzuP-ejzZdU-isLink-true', 'a[data-qa="author-name"]']
    authors = []
    for candidate in author_candidates:
        author_tags = soup.find_all(class_=candidate) or soup.find_all('a', {'data-qa': 'author-name'})
        for tag in author_tags:
            authors.append(tag.get_text(strip=True))
    author = ' and '.join(authors)

    # Get content
    content_list = soup.find_all(class_='article-body grid-center grid-body') or soup.find_all('p')
    content = ' '.join(content.get_text(strip=True) for content in content_list)

    # Get date
    date_string = None
    date_candidates = ['wpds-c-iKQyrV', 'wpds-c-iKQyrV wpds-c-iKQyrV-ihqANPJ-css overrideStyles']
    for candidate in date_candidates:
        date_tag = soup.find(class_=candidate)
        if date_tag:
            date_string = date_tag.get_text(strip=True)
            #date_string = date_string.replace('.m.', 'm').replace('EST', '').strip()
            break

    # Parse the date
    #date_format = "%B %d, %Y at %I:%M %p"
    #date = datetime.strptime(date_string, date_format) if date_string else None

    df_row['summary'] = summary
    df_row['author'] = author
    df_row['content'] = content
    #df_row['date'] = date
    df_row['date'] = date_string

    return df_row

## Scraper function to scrape articles from The New York Post
def scrape_ny_post_article(df_row):
    # Get soup
    soup = df_row['soup']

    # No summaries exist
    summary = None

    # Get author
    author_tag = soup.find('span', class_='meta__link')
    author = author_tag.get_text(strip=True)[16:] if author_tag else 'Unknown'

    # Get content
    content_list = soup.find_all('p')[1:-1]
    content = ' '.join(content.get_text(strip=True) for content in content_list)

    # Get date
    date_tag = soup.find('div', class_='date--updated__item')
    date_string = date_tag.find_all('span')[1].get_text(strip=True) if date_tag else 'Unknown'
    #date_string = date_string.replace('.m.', 'M').replace('ET', '').strip()
    #date_format = "%b. %d, %Y, %I:%M %p"
    #try:
        #date = datetime.strptime(date_string, date_format)
    #except ValueError:
        #date = 'Unknown'

    df_row['summary'] = summary
    df_row['author'] = author
    df_row['content'] = content
    #df_row['date'] = date
    df_row['date'] = date_string

    return df_row

## Scraper function to scrape articles from The Atlantic
def scrape_atlantic_article(df_row):
    # Get soup
    soup = df_row['soup']

    # Attempt to get summary
    summary_tag = soup.find(class_='ArticleHero_dek__EqdkK')
    summary = summary_tag.get_text(strip=True) if summary_tag else None

    # Attempt to get author
    author_tag = soup.find(class_='ArticleBylines_link__kNP4C')
    author = author_tag.get_text(strip=True) if author_tag else 'Unknown'

    # Attempt to get content
    content_tags = soup.find_all('p', class_='ArticleParagraph_root__4mszW')
    content = ' '.join(tag.get_text(strip=True) for tag in content_tags)

    # Attempt to get date
    date_tag = soup.find('time', class_='ArticleTimestamp_root__b3bL6')
    date = date_tag['datetime'] if date_tag else 'Unknown'

    # Parse the date if it's not 'Unknown'
    #if date != 'Unknown':
        #try:
            #date = datetime.strptime(date, '%Y-%m-%dT%H:%M:%SZ').isoformat()
        #except ValueError:
            #date = 'Unknown'

    # Update the df_row with the scraped data
    df_row['summary'] = summary
    df_row['author'] = author
    df_row['content'] = content
    df_row['date'] = date

    return df_row

## Scraper function to scrape articles from CNN
def scrape_cnn_article(df_row):
    # Get soup
    soup = df_row['soup']

    # No summaries for CNN articles
    summary = None

    # Check if it's a live news article and get the author, content, and date accordingly
    if '/live-news/' in df_row['url']:
        author_tag = soup.find('p', {'data-type': 'byline-area'})
        author = author_tag.get_text(strip=True) if author_tag else 'Unknown'

        content_tags = soup.find_all('p', class_='sc-gZMcBi render-stellar-contentstyles__Paragraph-sc-9v7nwy-2 dCwndB')
        content = ' '.join(tag.get_text(strip=True) for tag in content_tags)

        date_tag = soup.find('div', class_='hJIoKL')
        date = date_tag.get_text(strip=True) if date_tag else 'Unknown'
        date_string = ' '.join(date.split(' ')[-3:])+' '+str(date.split(' ')[2][1:3])+':'+(date.split(' ')[2][3:5]) if date!='Unknown' else 'Unknown'
    else:
        author_tag = soup.find(class_='byline__link')
        author = author_tag.get_text(strip=True) if author_tag else 'Unknown'

        content_tags = soup.find_all('p', class_='paragraph inline-placeholder')
        content = ' '.join(tag.get_text(strip=True) for tag in content_tags)

        date_tag = soup.find(class_='timestamp')
        #date_string = ' '.join(date_tag.get_text(strip=True).split(' ')[-7:][-3:]+date_tag.get_text(strip=True).split(' ')[-7:][0:2]) if date_tag else 'Unknown'
        date_string = date_tag.get_text(strip=True).split('\n')[-1] if date_tag else 'Unknown'

    # Parse the date if it's not 'Unknown'
    """
    if date_string != 'Unknown':
        try:
            if '/live-news/' in df_row['url']:
                date_format = "%B %d, %Y %H:%M"
            else:
                date_format = '%B %d, %Y %I:%M %p'
            date = datetime.strptime(date_string, date_format).isoformat()
        except ValueError:
            date = 'Unknown'
    """

    # Update the df_row with the scraped data
    df_row['summary'] = summary
    df_row['author'] = author
    df_row['content'] = content
    #df_row['date'] = date
    df_row['date'] = date_string

    return df_row

## Scraper function to scrape articles from Business Insider
def scrape_business_insider_article(df_row):
    # Get soup
    soup = df_row['soup']
    
    # No summary for BI
    summary = None

    # Attempt to get author
    author_tag = soup.find(class_='byline-author headline-bold')
    author = author_tag.get_text(strip=True) if author_tag else 'Unknown'

    # Attempt to get content
    content_tags = soup.find_all('p')
    content = ' '.join(tag.get_text(strip=True) for tag in content_tags[1:])

    # Attempt to get date
    date_tag = soup.find('div', class_='byline-timestamp')
    date = date_tag['data-timestamp'] if date_tag else 'Unknown'

    # Update the df_row with the scraped data
    df_row['summary'] = summary
    df_row['author'] = author
    df_row['content'] = content
    df_row['date'] = date

    return df_row

## Scraper function to scrape articles from Fox News
def scrape_fox_news_article(df_row):
    # Get soup
    soup = df_row['soup']

    # Attempt to get summary
    summary_tag = soup.find('h2', class_='sub-headline speakable')
    summary = summary_tag.get_text(strip=True) if summary_tag else None

    # Attempt to get author
    author_tag = soup.find(class_='author-byline')
    author = author_tag.get_text().split('\n')[-1].replace('Fox News','').strip() if author_tag else 'Unknown'

    # Attempt to get content
    content_tags = soup.find_all('p')
    content = ' '.join(tag.get_text(strip=True) for tag in content_tags[1:])

    # Attempt to get date
    date_tag = soup.find('time')
    #date_string = date_tag.get_text(strip=True)[:-4] if date_tag else 'Unknown'
    date_string = date_tag.get_text(strip=True) if date_tag else 'Unknown'

    """
    # Parse the date if it's not 'Unknown'
    if date_string != 'Unknown':
        try:
            date_format = "%B %d, %Y %I:%M%p"
            date = datetime.strptime(date_string, date_format)
        except ValueError:
            date = 'Unknown'
    else:
        date = 'Unknown'

    """

    # Update the df_row with the scraped data
    df_row['summary'] = summary
    df_row['author'] = author
    df_row['content'] = content
    #df_row['date'] = date
    df_row['date'] = date_string

    return df_row

In [3]:
class Top_News:
    # Class initializer with publication names and setup for scraping functions and class data
    def __init__(self, publication_names):
        self.publication_names = publication_names if publication_names != 'all' else ['NY Post', 'Atlantic', 'CNN', 'Business Insider', 'Washington Post', 'Fox News', 'Guardian']
        self.results_df = pd.DataFrame()
        # Dictionary containing scraping function for each publication
        self.scrapers = {
            'NY Post': scrape_ny_post_article,
            'Atlantic': scrape_atlantic_article,
            'CNN': scrape_cnn_article,
            'Business Insider': scrape_business_insider_article,
            'Washington Post': scrape_washington_post_article,
            'Fox News': scrape_fox_news_article,
            'Guardian': scrape_guardian_article
        }
        # Dictionary containing class data for each publication to locate articles on their homepage
        self.class_data = {
            'NY Post': ['https://nypost.com/', 'story__headline headline headline--xl', 'story__headline headline headline--sm', 'story__headline headline headline--combo-lg-xl headline--with-inline-webwood'],
            'Atlantic': ['https://www.theatlantic.com/world/', 'HomepageBottom_channelArticle__2wxRe', 'SmallPromoItem_root__nkm_2', 'Lede_title__7Wg1g', 'Offlede_title__kiinC', 'QuadBelt_title__mB6Zf', 'DoubleWide_title__diUPi', 'DoubleStack_title___FhPb', 'Latest_article__DW75m', 'Popular_listItem__CtMMj'],
            'CNN': ['https://edition.cnn.com/', 'container__link', 'container__title container_lead-package__title container__title--emphatic hover container__title--emphatic-size-l1'],
            'Business Insider': ['https://www.businessinsider.com/', 'tout', 'quick-link', 'most-popular-item', '.featured-tout-collection-wrapper .tout-title a', '.two-column-wrapper .tout-title-link'],
            'Washington Post': ['https://www.washingtonpost.com/', 'wpds-c-iiQaMf wpds-c-iiQaMf-igUpeQR-css', 'wpds-c-iiQaMf wpds-c-iiQaMf-ikZTsyd-css', 'wpds-c-iiQaMf wpds-c-iiQaMf-ibYgSwf-css'],
            'Fox News': ['https://www.foxnews.com/', 'article'],
            'Guardian': ['https://www.theguardian.com/', 'dcr-12ilguo', 'dcr-yw3hn9']
        }
    
    def _get_soup(self, url,rate_limit_seconds=1):
        try:
            # implement rate limiting for ethical reasons
            time.sleep(rate_limit_seconds)
            # try to get response from server and parse it into a BeautifulSoup object
            response = requests.get(url)
            response.raise_for_status()  
            return BeautifulSoup(response.text, 'html.parser')
        except requests.exceptions.RequestException as e:
            # handle exceptions
            print(f"Request failed for {url}: {e}")
            return None

    def scrape_publications(self):
        # use class data to find and parse articles from homepages
        selected_class_data = {name: self.class_data[name] for name in self.publication_names if name in self.class_data}
        articles_data = []
           
        for class_name in selected_class_data:
            # find articles and collect data
            base_url = selected_class_data[class_name][0]
            soup = self._get_soup(base_url)
            
            # find article blocks
            article_blocks = []
            for c_ in selected_class_data[class_name][1:]:
                temp = soup.find_all(class_=c_) if (class_name != 'Fox News') else soup.find_all('article')
                temp = temp if temp else soup.select(c_)
                article_blocks.extend(temp)
            
            # extract data from each block
            for block in article_blocks:
                url = block.find('a')['href'] if block.find('a') else (block['href'] if 'href' in block.attrs else None)
                if url:
                    article_data = {
                        'name': class_name,
                        'title': block.get_text(strip=True),
                        'url': base_url[:-1]+url if url.startswith('/') else url,
                        'soup': self._get_soup(url) # get content of article
                    }
                    articles_data.append(article_data)

        self.results_df = pd.DataFrame(articles_data)
        # drop duplicates
        self.results_df.drop_duplicates(subset=['url'], inplace=True)
        self.results_df.reset_index(drop=True, inplace=True)

        return self.results_df

    def article_distribution(self):
        if not self.results_df.empty:
            # plot distribution of articles from different publishers
            self.results_df.groupby('name').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
            plt.gca().spines[['top', 'right',]].set_visible(False)
            plt.show()
        else:
            print('Scrape publications first')

    def scrape_articles(self, max_articles_per_publication=None):
        if self.results_df.empty:
            print("Scraping publications for article URLs...")
            self.scrape_publications()

        all_article_data = []
        articles_count = {}

        for index, row in self.results_df.iterrows():
            publication_name = row['name']
            if publication_name in self.scrapers:
                articles_count.setdefault(publication_name, 0)
                # do not scrape more than the desired number of articles per publisher
                if max_articles_per_publication is None or articles_count[publication_name] < max_articles_per_publication:
                    try:
                        article_data = self.scrapers[publication_name](row)
                        all_article_data.append(article_data)
                        articles_count[publication_name] += 1
                    except Exception as e:
                        print(f"An error occurred while scraping {row['url']}: {e}")

        self.results_df = pd.DataFrame(all_article_data)
        self.results_df.reset_index(drop=True, inplace=True)

In [4]:
def scraper(filename='scraping_data.csv', publication_list='all', max_limit_num_articles=None):
    news_scraper = Top_News(publication_list)
    news_scraper.scrape_publications()
    #news_scraper.article_distribution()
    news_scraper.scrape_articles(max_articles_per_publication=max_limit_num_articles)
    filename = os.path.join('./data/', filename)
    news_scraper.results_df.to_csv(filename, index=False)

### Scrape the web:

In [5]:
# select_agencies = ['NY Post','Atlantic','CNN','Business Insider','Washington Post','Fox News','Guardian']
# number_of_articles = 10
# scraper(filename="new_scraping_data.csv", publication_list=select_agencies, max_limit_num_articles=int(number_of_articles))   

### TF-IDF class

In [6]:
class TFIDF:
    
    def __init__(self, df):
        self.df = df
        try:
            nltk.download('stopwords') # Download necessary NLTK resources to remove stop words
        except nltk.exceptions.AlreadyDownloaded:
            # Handle the case where the resource is already downloaded
            pass
    def preprocess_articles(self, text):
        text = text.lower() # Convert data to lover case to remove multiple occurences of the same word
        text = re.sub(r'[^a-z\s]', '', text) # Remove special characters, digits and white space using regex expression
        stop_words = set(stopwords.words('english')) # Import stop words from nltk library
        words = [word for word in text.split() if word not in stop_words] # Extract all words that are not stop words
        stemmer = PorterStemmer() # Apply stemming using nltk PorterStemmer
        stemmed_words = [stemmer.stem(word) for word in words]
        return ' '.join(stemmed_words)

    # Create vocabulary of words from news articles
    def vocabulary(self, docs):
        vocab = set()
        for doc in docs:
            for word in doc:
                vocab.add(word)
        return sorted(vocab)

    # Calculate Term frequency
    def term_frequency(self, docs, vocab):
        tf = pd.DataFrame(0, index=range(len(docs)), columns=vocab)
        for i, document in enumerate(docs):
            num_words_doc = len(document)
            for word in document:
                tf.at[i, word] += document.count(word)/num_words_doc
        return tf

    # Inverse Document Frequency
    def inverse_document_frequency(self, docs, vocab):
        # We want to reduce the weight of terms that appear frequently in our collection of articles.
        idf = pd.Series(0, index=vocab) # Create series and set all elements to 0
        for word in vocab:
            counter = 0
            for doc in docs:
                if word in doc:
                    counter +=1
            idf[word] = np.log((len(docs))/(counter+1)) #TFIDF as stated in the slides of week 1
        return idf
    
    # TF-IDF
    def tf_idf(self, tf, idf, docs, vocab):
        tfidf = pd.DataFrame(index=range(len(docs)), columns=vocab)  # Create an empty DataFrame
        for i in range(len(docs)):
            tfidf.iloc[i] = tf.iloc[i]*idf  # Multiply TF values by IDF for each term
        tfidf = normalize(tfidf, norm='l2', axis=1)  # L2 normalization to scale vectors
        tfidf = pd.DataFrame(tfidf, columns=vocab)
        return tfidf

    def fit(self):
        # Apply preprocesing to news articles
        self.df['preprocessed_content'] = self.df['content'].apply(self.preprocess_articles)
        # Extract words from each news articles
        docs = self.df['preprocessed_content'].str.split()
        # Create a vocabulary corresponding to all the words in every news article
        vocab = self.vocabulary(docs)
        # Calculate the TF
        tf = self.term_frequency(docs, vocab)
        # Calculate the IDF
        idf = self.inverse_document_frequency(docs, vocab)
        # Multiply TF with IDF and calculate TF-IDF
        tfidf = self.tf_idf(tf, idf, docs, vocab)
        return tfidf

### K-means clustering class

In [7]:
class KMeansAlgorithm:

    def __init__(self, X, k, max_iterations=100):
      self.X = X.values
      self.k = k
      self.max_iterations = max_iterations
      self.centroids = None
      self.num_articles = X.shape[0]

    def init_random_centroids(self):
      # Initializing KMeans by choosing random centroids 
      idx = np.random.choice(self.num_articles, size=self.k, replace=False) # Extract random indices from dataframe
      centroids = self.X[idx] # Pick random rows from the dataframe
      return centroids

    def calculate_euclidean_distances(self):
      # Calculate distances from vector/row to centroid
      num_centroids = self.centroids.shape[0]
      distances = np.zeros((num_centroids, self.num_articles))

      for centroid_idx in range(num_centroids):
          for article_idx in range(self.num_articles):
              distances[centroid_idx, article_idx] = np.sqrt(np.sum((self.centroids[centroid_idx, :] - self.X[article_idx, :]) ** 2))
      return distances

    def update_centroids(self, labels):
        # Calculate the mean of each cluster as new centroid
        new_centroids = []
        for k in range(self.k):
            mean = self.X[labels == k].mean(axis=0)
            new_centroids.append(mean)
        new_centroids = np.array(new_centroids)
        return new_centroids

    def plot_clusters(self, labels, centroids, iteration):
        # We will use PCA to plots clusters as the TFIDF matrix has many dimensions
        unique_labels = np.unique(labels)
        pca = PCA(n_components=3)
        data_3d = pca.fit_transform(self.X)
        centroids_3d = pca.transform(centroids)
        # 3D plot
        fig = plt.figure(figsize=(8, 6))
        ax = fig.add_subplot(111, projection='3d')
        color_map = cm.get_cmap('turbo', len(unique_labels))
        # Plot data
        for label in unique_labels:
            indices = labels == label # like [1, 2, 1, 3, 1]
            ax.scatter(data_3d[indices, 0], data_3d[indices, 1], data_3d[indices, 2], label=f'Cluster {label}', c=[color_map(label)])
        # Plot centroids
        ax.scatter(centroids_3d[:, 0], centroids_3d[:, 1], centroids_3d[:, 2], c='red', marker='x', s=80, label='Centroids')
        ax.set_title(f'PCA 3D - iteration {iteration}')
        ax.set_xlabel('Principal component 1')
        ax.set_ylabel('Principal component 2')
        ax.set_zlabel('Principal component 3')
        ax.legend(loc='center left', bbox_to_anchor=(1.1, 0.5))
        clear_output(wait=True)
        plt.show()

    def fit(self):
      print("Clustering: KMeans fit()", datetime.now().strftime("%H:%M:%S"))
      
      # Run the Kmeans algorithm using helper functions
      self.centroids = self.init_random_centroids() # Init centroids

      for i in range(self.max_iterations):
        distances = self.calculate_euclidean_distances() # calculate eucledian distances
        labels = np.argmin(distances, axis=0) # Assign to the cluster with the centroid that has the minimum distance to that point
        new_centroids = self.update_centroids(labels) # Calculated centroids based on mean of the points in that cluster

        if np.all(new_centroids == self.centroids): # If no new centroids break loop
          print("Clustering: Kmeans has converged!", datetime.now().strftime("%H:%M:%S"))
          break

        self.centroids = new_centroids
        self.plot_clusters(labels, self.centroids, i) # Plot PCA 3D plot

      return labels

In [8]:
def extract_documents(df):
    """
    Main TF-IDF Function
    Total time for 500 articles: 20 minutes

    Args:
        df (dataframe): dataframe with preprocessed content
        
    Returns:
        tf idf matrix
    """
    tfidf = TFIDF(df)
    tfidf_matrix = tfidf.fit()

    return tfidf_matrix

def Cluster_Articles(k, tfidf_matrix):
    tfidf_matrix = extract_documents(tfidf_matrix)
    Kmeans = KMeansAlgorithm(tfidf_matrix, k) # Optimal number of clusters is determined from the results of the elbow method
    labels = Kmeans.fit()
    tfidf_matrix['cluster'] = labels
    return tfidf_matrix

def elbow_method(data):
    # We will use elbow method to determine optimal number of clusters
    num_clusters = range(1, 30)
    wcss = []

    for k in num_clusters:
        # We will use Sklearn's Kmeans algorithm, but use random intialization as used in our implementation
        kmeans = KMeans(n_clusters=k, init='random', random_state=123)
        kmeans.fit(data)
        wcss.append(kmeans.inertia_)

    # Plot the Elbow Method
    plt.plot(num_clusters, wcss, marker='o')
    plt.title('Elbow Method - KMeans with random initialization')
    plt.xlabel('Number of clusters')
    plt.ylabel('Wcss')
    plt.show()

### Load datasets to analyze

In [9]:
# dataset_paths = [file for file in os.listdir("./data/") if file.endswith(('csv'))] # Select all available data files
# dataset_paths = ["scraping_data_filtered.csv"]
dataset_paths = ["articles1.csv"]

# Load data
data_frames = []
data = []
DATA_LIMIT = 50 #520 # how many items to load
for data_path in dataset_paths:
    # data_frame_temp = pd.read_csv(os.path.join('./data/', data_path)).head(DATA_LIMIT)
    # print(data_frame_temp.shape[0])
    # for i in range(data_frame_temp.shape[0]):
        #   data_frame_temp.loc[i,'content'] = str(data_frame_temp.loc[i,'content'])
    # data_frame_temp['content'] = data_frame_temp['content'].astype(str)
    data_frames.append(pd.read_csv(os.path.join('./data/', data_path)).head(DATA_LIMIT))
    # for i in range(data)
    # data_frames['content'] = data_frames['content'].astype(str)
    # data_frames.append(data_frame_temp)
data = pd.concat(data_frames)
print(type(data))
data['date'] = pd.to_datetime(data['date'],format='mixed')
data = data.drop(['Unnamed: 0', 'year', 'month', 'url'], axis=1, errors='ignore')

if len(data) > 0:
        # display dataframe header
        print("First 20 entries in data (size " + str(len(data)) + "):")
        display(data.head(20))

<class 'pandas.core.frame.DataFrame'>
First 20 entries in data (size 50):


,id,title,publication,author,date,content
0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,WASHINGTON — Congressional Republicans have...
1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,"After the bullet shells get counted, the blood..."
2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,"When Walt Disney’s “Bambi” opened in 1942, cri..."
3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,"Death may be the great equalizer, but it isn’t..."
4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,"SEOUL, South Korea — North Korea’s leader, ..."
5,17288,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,"LONDON — Queen Elizabeth II, who has been b..."
6,17289,Taiwan’s President Accuses China of Renewed In...,New York Times,Javier C. Hernández,2017-01-02,BEIJING — President Tsai of Taiwan sharpl...
7,17290,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,"Danny Cahill stood, slightly dazed, in a blizz..."
8,17291,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,"Just how is Hillary Kerr, the founder of ..."
9,17292,Calling on Angels While Enduring the Trials of...,New York Times,Andy Newman,2016-12-31,Angels are everywhere in the Muñiz family’s ap...


## Running TF-IDF

### Process Similar Articles

In [10]:
nr_clusters = 8 # Number of clusters
    
clustered_data = pd.DataFrame()

test = data.copy()
print(type(test))

data = Cluster_Articles(k=int(nr_clusters), data=data.copy())
    
if 'cluster' in data:
    # display dataframe header
    print("First 20 entries in data with clusters:")
    data[['title', 'cluster']].head(5)
    
    # Save clusters
    # clustered_path = text_area("Path", value="clustered_articles1.csv")
    # data.to_csv(os.path.join("data/", str(clustered_path)))

<class 'pandas.core.frame.DataFrame'>


TypeError: Cluster_Articles() got an unexpected keyword argument 'data'